## Youtube comments spam detection using auto-label

Import libraries and Setup the API keys:

In [1]:
import os
import pandas as pd
import evadb
import base64

encoded_open_api_key = "c2stUGhReTViUVVZMVBjQ2lKQ2swTE5UM0JsYmtGSmliR0syTE9lSzlTV2hLeWFubE1o"
decoded_bytes = base64.b64decode(encoded_open_api_key)
os.environ['OPENAI_API_KEY'] = decoded_bytes.decode('utf-8')

Declare csv file:

In [2]:
csv_file = "spam-ham-label/data/test.csv"
  

In [3]:
cursor = evadb.connect().cursor()
print("Connected to EvaDB")

Connected to EvaDB


In [4]:
create_function_query = f"""CREATE FUNCTION IF NOT EXISTS AutoLabel
            IMPL  './functions/autolabel.py';
            """
cursor.query("DROP FUNCTION IF EXISTS AutoLabel;").execute()
cursor.query(create_function_query).execute()
print("Created Function")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS YTCOMMENTS(
comment_id TEXT(200),
author TEXT(30),
date TEXT(10),
content TEXT(255)
);
"""

load_data_query = f""" LOAD CSV 'spam-ham-label/data/test.csv' INTO YTCOMMENTS;""" 

12-18-2023 00:05:55 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0089] Function AutoLabel does not exist, therefore cannot be dropped.


Created Function


Create table and load data:

In [5]:
cursor.query(create_table_query).execute()
cursor.query(load_data_query).execute()

Check cost of Data Labeling:

In [6]:
query= f""" SELECT AutoLabel("plan", comment_id, author, date, content) FROM YTCOMMENTS;"""

result = cursor.query(query).execute()

c:\Users\pmurthy34\Documents\evadb-auto-label\dataframe\labeling_agent.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.df = df


Total cost of labeling would be: $ 0.00179


Perform Data Labeling:

In [7]:
query= f""" SELECT AutoLabel("run", comment_id, author, date, content) FROM YTCOMMENTS;"""

result = cursor.query(query).execute()
print(result)

c:\Users\pmurthy34\Documents\evadb-auto-label\dataframe\labeling_agent.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.df = df


Now I want you to label the following comments:
Input: Please go in and see our Channel and subscribe :-). It would be Nice mate.
Input: I swear Shakira keeps getting more and more gorgeous! She definitely looks more gorgeous with her hair this way than super curly.﻿
Input: Please go in and see our Channel and subscribe :-). It would be Nice mate.
Input: Hey guys whats up? I found this app that lets you get free gift card vouchers like psn cards,X-box live cards and even amazon gift cards. For free! All you have to do is  simply just download the app from the app store. It is called juno wallet. All you have to do is just sign up for the app and then complete a few surveys or just download some other free apps and you get money like 10 cents. Also, if you type in the code IM2458444. You will also start off with $0.25 free!! 
Input: &quot;HELP THE HUMANITY WITH YOUR SIGN IN TO THIS LINK WITH YOUR WHOLEHEARTED SUPPORT IF YOU SAY &quot;NO&quot; TO DISCRIMINATION. “UNITED WE STAND “  WE WI

Validate results

In [8]:
import pandas as pd

def compare_csv_files(test_csv, validate_csv):
    label_df = pd.read_csv(test_csv)
    ground_truth = pd.read_csv(validate_csv)
    are_equal = (label_df['class'] == ground_truth['class']).all()
    return are_equal

test_csv = 'spam-ham-label/data/labeled_data.csv'
validate_csv = 'spam-ham-label/data/validate.csv'

if compare_csv_files(test_csv, validate_csv):
    print("The labeled data CSV is the same as the validate CSV.")
else:
    print("The labeled data CSV is different from the validate CSV.")


The labeled data CSV is the same as the validate CSV.
